In [4]:
!python -m pip install https://github.com/schmiph2/pysepm/archive/master.zip

  Using cached https://github.com/schmiph2/pysepm/archive/master.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://github.com/ludlows/python-pesq/archive/master.zip
  Preparing metadata (setup.py) ... done
  Using cached https://github.com/jfsantos/SRMRpy/archive/master.zip
  Preparing metadata (setup.py) ... done
  Using cached https://github.com/detly/gammatone/archive/master.zip
  Preparing metadata (setup.py) ... done


In [5]:
import pysepm

In [6]:
import torch
import torchaudio
from denoiser import pretrained
from denoiser.dsp import convert_audio


In [7]:
import os
from glob import glob

In [8]:
# from scipy.io import wavfile
# # import sys
# # sys.path.append("../") 



# Load Audio Files

In [9]:
DATA_DIR = "/home/ubuntu/efs/IntroDL/denoiser/dataset"
CLEAN_DIR = os.path.join(DATA_DIR, "starter_clean/clean")
NOISY_DIR = os.path.join(DATA_DIR, "starter_noisy/noisy")



In [10]:
clean_files = sorted(glob(CLEAN_DIR + "/*"))
noisy_files = sorted(glob(NOISY_DIR + "/*"))

`clean_files` and `noisy_files` does not have same order of files ; i.e. the i-th file of `clean_files` does not correspond to the i-th files of `noisy_files`. Hence, we should make the file id of the i-th file of `clean_files` and `noisy_files` correspond to each other. 

In [11]:
i = 100
name = os.path.basename(noisy_files[i]).split('_')[-1]
print(name)

142.wav


In [12]:
clean_dict = {os.path.basename(name).split('_')[-1]:name for name in clean_files}
noisy_dict = {os.path.basename(name).split('_')[-1]:name for name in noisy_files}


In [13]:
print(len(clean_files) == len(noisy_files))
print(set(clean_dict.keys()) == set(noisy_dict.keys()))
# Check that the set of file id of clean files and noisy files are exactly the same

True
True


In [14]:
clean_files = []
noisy_files = []
for fileid in clean_dict.keys() : 
    clean_files.append(clean_dict[fileid])
    noisy_files.append(noisy_dict[fileid])


In [15]:
print(noisy_files[:5])
print(clean_files[:5])

['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp126_3Wjw0nadnM4_snr15_tl-22_fileid_0.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp198_bus_56903_0_snr0_tl-32_fileid_101.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp535_washer_460549_1_snr2_tl-35_fileid_102.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp228_vacuum_379760_1_snr1_tl-19_fileid_104.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp591_clatter_noise_166453__zabuhailo__metalworking-hammer-5_5_snr9_tl-27_fileid_105.wav']
['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_0.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_101.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_102.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_104.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/s

In [16]:
print(noisy_files[-5:])
print(clean_files[-5:])

['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp644_barking_25887_0_snr5_tl-28_fileid_89.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp81_car_74675_2_snr18_tl-22_fileid_9.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp177_0kuIP7fLNpM_snr6_tl-26_fileid_90.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp383_babble_188218_15_snr6_tl-18_fileid_94.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp219_wind_344887_0_snr10_tl-28_fileid_96.wav']
['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_89.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_9.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_90.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_94.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_96.wav']


We have just checked that the order of files in the list are matched for `clean_files` and `noisy_files`.

In [17]:
clean_wav, clean_sr = torchaudio.load(clean_files[0])
noisy_wav, noisy_sr = torchaudio.load(noisy_files[0])

In [18]:
clean_wav

tensor([[ 0.0731,  0.0881,  0.1040,  ..., -0.0025, -0.0023, -0.0017]])

In [19]:
noisy_wav

tensor([[0.0877, 0.1019, 0.1178,  ..., 0.0010, 0.0010, 0.0011]])

In [20]:
print(clean_sr)
print(noisy_sr)

16000
16000


In [21]:
clean_wav.shape

torch.Size([1, 160000])

Note that in the [example notebook](https://github.com/schmiph2/pysepm/blob/master/examples/examplesForCalculatingMeasures.ipynb) provided by the github repository of **pysepm**, the type and the shape of the input are given as numpy array and $(N,)$ where $N$ is equal to the product of `sample_rate` and `duration of audio file (in seconds)`

In [22]:
clean_wav = clean_wav.numpy().reshape(-1)
noisy_wav = noisy_wav.numpy().reshape(-1)

In [23]:
print(type(clean_wav))
print(clean_wav.shape)

<class 'numpy.ndarray'>
(160000,)


Bring the file list of enhanced audio files in the same order as the clean and noisy files list.

In [24]:
RESULT_DIR = "/home/ubuntu/efs/IntroDL/denoiser/results/Demucs_denoised"

enhanced_files = sorted(glob(RESULT_DIR + "/*"))
enhanced_dict = {os.path.basename(name).split('_')[-1]:name for name in enhanced_files}



In [25]:
enhanced_files = []
for fileid in clean_dict.keys() : 
    enhanced_files.append(enhanced_dict[fileid])

In [26]:
print(noisy_files[:5])
print(clean_files[:5])
print(enhanced_files[:5])

['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp126_3Wjw0nadnM4_snr15_tl-22_fileid_0.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp198_bus_56903_0_snr0_tl-32_fileid_101.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp535_washer_460549_1_snr2_tl-35_fileid_102.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp228_vacuum_379760_1_snr1_tl-19_fileid_104.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp591_clatter_noise_166453__zabuhailo__metalworking-hammer-5_5_snr9_tl-27_fileid_105.wav']
['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_0.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_101.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_102.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_104.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/s

In [27]:
enhanced_wav, enhanced_sr = torchaudio.load(enhanced_files[0])

In [28]:
enhanced_wav

tensor([[ 0.0834,  0.0984,  0.1180,  ..., -0.0008, -0.0008, -0.0006]])

In [29]:
enhanced_sr

16000

In [30]:
enhanced_wav = enhanced_wav.numpy().reshape(-1)


# Calc Measures

## STOI

In [31]:
pysepm.stoi(clean_wav, noisy_wav, clean_sr)

0.9807254867015377

In [32]:
pysepm.stoi(clean_wav, enhanced_wav, clean_sr)

0.9933100283139101

Conjecture by Taeyoung

STOI :  
- somehow measures the correlation between two waveforms
- the higher the better

## PESQ

In [33]:
pysepm.pesq(clean_wav, noisy_wav, clean_sr)[1]

2.3495934009552

In [34]:
pysepm.pesq(clean_wav, enhanced_wav, clean_sr)[1]

3.5990958213806152

Conjecture by Taeyoung

PESQ :  
- Subtract disturbance from $a_0 = 4.5$
- Closer to 4.5 the better

## fwSNRseg

In [35]:
pysepm.fwSNRseg(clean_wav, noisy_wav, clean_sr)

20.69275673110846

In [36]:
pysepm.fwSNRseg(clean_wav, enhanced_wav, clean_sr)

22.317664763495486

Conjecture by Taeyoung

Signal to noise ratio :  
- first argument is clean, which is signal 
- the second argument is noisy, which is signal + noise.
- Noise should be small, so high SNR is desired.

In [37]:
STOI_noisy = []
STOI_enhanced = []
PESQ_noisy = []
PESQ_enhanced = []
FWSNR_noisy = []
FWSNR_enhanced = []

for i in range(4, len(clean_files)) :
    clean_wav, clean_sr = torchaudio.load(clean_files[i])
    noisy_wav, noisy_sr = torchaudio.load(noisy_files[i])
    enhanced_wav, enhanced_sr = torchaudio.load(enhanced_files[i])
    
    clean_wav = clean_wav.numpy().reshape(-1)
    noisy_wav = noisy_wav.numpy().reshape(-1)
    enhanced_wav = enhanced_wav.numpy().reshape(-1)

    STOI_noisy.append(pysepm.stoi(clean_wav, noisy_wav, clean_sr))
    STOI_enhanced.append(pysepm.stoi(clean_wav, enhanced_wav, clean_sr))

    PESQ_noisy.append(pysepm.pesq(clean_wav, noisy_wav, clean_sr)[1])
    PESQ_enhanced.append(pysepm.pesq(clean_wav, enhanced_wav, clean_sr)[1])

    FWSNR_noisy.append(pysepm.fwSNRseg(clean_wav, noisy_wav, clean_sr))
    FWSNR_enhanced.append(pysepm.fwSNRseg(clean_wav, enhanced_wav, clean_sr))




In [56]:
import numpy as np

In [58]:
print("mean and stdev of STOI for noisy files are {:.4f} , {:.4f}".format(np.mean(STOI_noisy),np.std(STOI_noisy)))
print("mean and stdev of STOI for enhanced files are {:.4f} , {:.4f}".format(np.mean(STOI_enhanced),np.std(STOI_enhanced)))
print("mean and stdev of PESQ for noisy files are {:.4f} , {:.4f}".format(np.mean(PESQ_noisy),np.std(PESQ_noisy)))
print("mean and stdev of PESQ for enhanced files are {:.4f} , {:.4f}".format(np.mean(PESQ_enhanced),np.std(PESQ_enhanced)))
print("mean and stdev of fwSNRseg for noisy files are {:.4f} , {:.4f}".format(np.mean(FWSNR_noisy),np.std(FWSNR_noisy)))
print("mean and stdev of fwSNRseg for enhanced files are {:.4f} , {:.4f}".format(np.mean(FWSNR_enhanced),np.std(FWSNR_enhanced)))


mean and stdev of STOI for noisy files are 0.9154 , 0.0651
mean and stdev of STOI for enhanced files are 0.9650 , 0.0332
mean and stdev of PESQ for noisy files are 1.5863 , 0.4547
mean and stdev of PESQ for enhanced files are 2.6500 , 0.6314
mean and stdev of fwSNRseg for noisy files are 12.6105 , 4.9824
mean and stdev of fwSNRseg for enhanced files are 17.1134 , 3.7672


## SNRseg

In [5]:
pysepm.SNRseg(clean_speech, noisy_speech, fs)

-0.21686174881958412

In [6]:
pysepm.SNRseg(clean_speech, enhanced_speech, fs)

-1.2033705198493232

## LLR

In [7]:
pysepm.llr(clean_speech, noisy_speech, fs)

1.2530765533868626

In [8]:
pysepm.llr(clean_speech, enhanced_speech, fs)

1.6044180854910925

## WSS

In [9]:
pysepm.wss(clean_speech, noisy_speech, fs)

44.69861369131994

In [10]:
pysepm.wss(clean_speech, enhanced_speech, fs)

66.79548150633117

## Cepstrum Distance (CD)

In [11]:
pysepm.cepstrum_distance(clean_speech, noisy_speech, fs)

7.187803223674154

In [12]:
pysepm.cepstrum_distance(clean_speech, enhanced_speech, fs)

8.308018500622689

## CSII

In [15]:
pysepm.csii(clean_speech, noisy_speech, fs)

(0.7336988398405825, 0.4363089356143529, 0.018848854017316796)

In [16]:
pysepm.csii(clean_speech, enhanced_speech, fs)

(0.527623436040412, 0.2437425452887004, 0.010118033728508934)

## Composite

In [19]:
pysepm.composite(clean_speech, noisy_speech, fs)

(2.037992006581127, 1.8630831647556954, 1.5433156477547219)

In [20]:
pysepm.composite(clean_speech, enhanced_speech, fs)

(1.0, 1.5970461451303148, 1.0)

## NCM

In [21]:
pysepm.ncm(clean_speech, noisy_speech, fs)

In [22]:
pysepm.ncm(clean_speech, enhanced_speech, fs)

# SRMR

In [23]:
pysepm.srmr(noisy_speech, fs)

/usr/local/lib/python3.6/dist-packages/srmrpy/hilbert.py:69: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  h = h[ind]


3.2050299299144114

In [24]:
pysepm.srmr(enhanced_speech, fs)

7.317370449764798

# BSD

In [10]:
pysepm.bsd(clean_speech, noisy_speech, fs)

include pre-emphasis


38086949616.83964

In [11]:
pysepm.bsd(clean_speech, enhanced_speech, fs)

include pre-emphasis


51047508.139671564

# Measure Execution Times

In [25]:
%timeit pysepm.fwSNRseg(clean_speech, noisy_speech, fs)

61.1 ms ± 667 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%timeit pysepm.SNRseg(clean_speech, noisy_speech, fs)

14.5 ms ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
%timeit pysepm.llr(clean_speech, noisy_speech, fs)

113 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%timeit pysepm.wss(clean_speech, noisy_speech, fs)

80 ms ± 552 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%timeit pysepm.cepstrum_distance(clean_speech, noisy_speech, fs)

92.5 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%timeit pysepm.stoi(clean_speech, noisy_speech, fs)

143 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%timeit pysepm.csii(clean_speech, noisy_speech, fs)

82.7 ms ± 450 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit pysepm.pesq(clean_speech, noisy_speech, fs)

309 ms ± 510 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit pysepm.composite(clean_speech, noisy_speech, fs)

521 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit pysepm.ncm(clean_speech, noisy_speech, fs)

5.13 s ± 13.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit pysepm.srmr(clean_speech, fs)

/usr/local/lib/python3.6/dist-packages/srmrpy/hilbert.py:69: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  h = h[ind]


1.6 s ± 660 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit pysepm.bsd(clean_speech, noisy_speech, fs)

include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter not tested
include pre-emphasis
bark filter